In [36]:
import os
import inspect
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from rasp_generator import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUT = [1,2,3,4]

# Program Generation

In [ ]:
# Remaining problems
# - sometimes categorical Aggregate is hard to sample (reaches max retries)
# - sometimes a sampled program doesn't depend on rasp.tokens

In [28]:
sampler = sampling.ProgramSampler(validate_compilation=True)
sampler.sample()
print(sampler.output([1,2,3,4]))

cat
cat


ValueError: Unsupported type for aggregation: [0.0, 1.0, 3.0, 6.0]

In [32]:
sampler.sops

[tokens,
 indices,

In [37]:
inspect.getsource(sampler.sops[-2].f)

'      self.f = lambda t: f(inner_f(t))\n'

In [38]:
# print program with and without test input
k = 2
utils.print_program(sampler.sops[-k])
print()
print()
utils.print_program(sampler.sops[-k], TEST_INPUT)

sequence_map_52 = SequenceMap(lambda x, y: x*y, tokens, indices)    # type: categorical
map_53 = Map(<function Map.__init__.<locals>.<lambda> at 0x7fdf688885e0>, sequence_map_52)    # type: categorical


sequence_map_52 = SequenceMap(lambda x, y: x*y, tokens, indices)    # output: [0, 2, 6, 12]
map_53 = Map(<function Map.__init__.<locals>.<lambda> at 0x7fdf688885e0>, sequence_map_52)    # output: [0.0, 1.0, 3.0, 6.0]


In [30]:
# print program with and without test input
utils.print_program(sampler.sops[2])
print()
print()
utils.print_program(sampler.sops[2], TEST_INPUT)

sequence_map_52 = SequenceMap(lambda x, y: x*y, tokens, indices)    # type: categorical


sequence_map_52 = SequenceMap(lambda x, y: x*y, tokens, indices)    # output: [0, 2, 6, 12]


In [ ]:
def get_program():
    select_22 = Select(indices, tokens, predicate=Comparison.FALSE)
    aggregate_21 = Aggregate(select_22, tokens)    # type: categorical
    return aggregate_21


program = get_program()
model = compile_rasp_to_model(program)


print(program(TEST_INPUT))
print(model.apply(["BOS", *TEST_INPUT]).decoded)

[None, None, None, None]
['BOS', 1, 1, 1, 1]
